# Colab files inclusion


In [ ]:
!pip install import-ipynb
import import_ipynb

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Question Answerer Inclusion


In [4]:
question_answerer_module = drive.CreateFile({"id": "YOUR_QUESTION_ANSWERER_FILE_ID"})
question_answerer_module.GetContentFile("question_answerer.ipynb")
import question_answerer


### Sentence Similarity Inclusion

In [ ]:
sentence_similarity_module = drive.CreateFile({'id':'YOUR_SENTENCE_SIMILARITY_FILE_ID'})
sentence_similarity_module.GetContentFile('sentence_similarity.ipynb')
import sentence_similarity

### Find Recipes Inclusion

In [ ]:
find_recipes_module = drive.CreateFile({'id':'YOUR_FIND_RECIPES_FILE_ID'})
find_recipes_module.GetContentFile('find_recipes.ipynb')
import find_recipes

# Prerequisites

In [ ]:
!pip install discord
!pip install python-dotenv
!pip install asyncio

### Morfeusz

In [9]:
import morfeusz2
morf = morfeusz2.Morfeusz()

# Imports

In [10]:
import asyncio
import os
import re
import random
import discord
from discord import channel
from dotenv import load_dotenv
from typing import List, Tuple, Union, Any

# Source

## environmental variables

In [32]:
os.environ["DISCORD_TOKEN"] = "YOUR_BOT_TOKEN"

load_dotenv()
TOKEN = os.getenv("DISCORD_TOKEN")

client = discord.Client()


## question_answerer helpers

In [33]:
def is_question(message_input: str) -> bool:
    """Determine if a message is a question or not."""
    question_starters = [
        "co",
        "ile",
        "dla kogo",
        "kto",
        "gdzie",
        "czy",
        "jak",
        "skąd",
        "po co",
        "kiedy",
        "komu",
        "czym",
        "czemu",
        "kogo",
        "dlaczego",
    ]
    return (
        message_input.endswith("?")
        and message_input.startswith(tuple(question_starters))
    ) or message_input.endswith("?")


In [34]:
def take_into_consideration() -> str:
    """Help with answering questions in more human like way."""
    wondering_phrases = [
        "Hmmm, niech no pomyślę..",
        "Daj mi chwilkę, zaraz odpowiem.",
        "Acha! Takiego pytania się nie spodziewałem. Już sobie przypominam...",
        "Ciekawe, ciekawe, może to...",
        "Sekundkę!",
        "Hmmmmm....",
        "Chwila!",
        "Trudne pytanie, muszę sprawdzić encyklopedię.",
        "Hmm",
    ]
    return random.choice(wondering_phrases)


In [35]:
def more_human_like_answer(answer: str) -> str:
    """Prettify answer returned by question answerer to act like a human."""
    answer = answer.split(" ")
    prettifiers = {
        "result_phrases": [
            "",
            "",
            "",
            "",
            "Zdaje się, że to ",
            "Wydaje mi się, że to ",
            "Nie jestem do końca pewien, ale czy to ",
            "Stawiam, że to ",
            "Czy mam rację i to ",
            "Hmmm, może to ",
            "Jestem prawie pewien, że to ",
        ],
        "ending_phrases": [
            ".",
            ".",
            ".",
            ".",
            ".",
            ". To było oczywiste!",
            ". Wiadomo!",
            ". Mam rację?",
            ". Mamma mia! To było trudne.",
        ],
    }
    if len(answer) == 1:
        end_or_begining = random.choice(list(prettifiers.keys()))
        if end_or_begining == "result_phrases":
            return random.choice(prettifiers["result_phrases"]) + " ".join(answer)
        return " ".join(answer) + random.choice(prettifiers["ending_phrases"])
    answer = " ".join(answer)
    answer = answer.capitalize()
    return answer + random.choice(prettifiers["ending_phrases"])


In [36]:
def answer_the_question(
    question: str,
    times_of_execution: int = 1,
) -> str:
    """Answer the question depending on time of execution."""
    return (
        take_into_consideration()
        if times_of_execution == 0
        else more_human_like_answer(question_answerer.find_answer(question))
    )


## utilities

### finding_subjects and verbs

In [37]:
def find_all_subjects_and_verbs(sentence: str) -> Tuple[List[str], List[str], bool]:
    """Find all subjects and verbs in a sentence"""
    analysis = morf.analyse(sentence)
    subjects = set()
    verbs = set()
    is_sec_sig = False
    verbs_tags = [
        "verb",
        "refl",
        "nonrefl",
        "perf",
        "imperf",
        "imperf.perf",
        "praet",
        "inf",
        "fin",
    ]
    for i, j, interp in analysis:
        word_tags = interp[2].split(":")
        first_tag = word_tags[0]
        if first_tag == "subst":
            subjects.add(interp[0])
        elif first_tag in verbs_tags:
            verbs.add(interp[0])
        if "sg" in word_tags and "sec" in word_tags:
            is_sec_sig = True
    return list(subjects), list(verbs), is_sec_sig


### greet handling

In [38]:
powitania = [
    "cześć",
    "hej",
    "witam",
    "siemka",
    "dzień dobry",
    "elo",
    "siemano",
    "Hejka naklejka",
    "Siemano, uważaj na kolano",
    "Uszanowanie, poproszę o jedno pytanie",
    "Elo Pomelo",
    "Serdecznie witam i szybko czytam",
    "Siemaneczko prawilna mordeczko",
]


def is_greeting(message_input: str) -> bool:
    """Determine if a message is a greeting."""
    return message_input.startswith(tuple(powitania))


def greet() -> str:
    """A function to greet the user."""
    return (random.choice(powitania)).capitalize()


def process_greeting(input_message: str, is_sec_sig: bool, words: List[str]) -> str:
    """Process users greeting to greet him back."""
    input_splitted = input_message.split()
    if len(input_splitted) > 1:
        rest_of_message = " ".join(input_splitted[1:])
        if discern_personal_messages(rest_of_message):
            return discern_personal_messages(rest_of_message)
        if not is_sec_sig and is_question(rest_of_message):
            return greet() + ". " + answer_the_question(rest_of_message)
        else:
            return (
                greet()
                + ". "
                + (
                    sentence_similarity.generate_answer(" ".join(input_splitted[1:]))
                ).capitalize()
            )
    else:
        return greet()


### tokenizing

In [39]:
polish_signs_replacements = {
    "ą": "a",
    "ć": "c",
    "ę": "e",
    "ł": "l",
    "ń": "n",
    "ó": "o",
    "ś": "s",
    "ź": "z",
    "ż": "z",
}


def tokenize_word(word: str) -> str:
    """Take word and replace polish signs with letters easy to replace them.
    Eq.: ą -> a, ę -> e
    """
    for polish_letter in polish_signs_replacements.keys():
        if polish_letter in word:
            word = word.replace(polish_letter, polish_signs_replacements[polish_letter])
    return word


def tokenize(message: str) -> str:
    """Take message and tokenize it to get rid of polish signs."""
    text = message.lower()
    pattern = r"\s*\w*\s*"
    result = re.findall(pattern, text)
    tokenized_words = [tokenize_word(word) for word in result]
    return "".join(tokenized_words)


### personal information

In [40]:
personal_data = {
    "imię": "Panda",
    "nazwisko": "Gruba",
    "wiek": "23",
    "ulubione_potrawy": ["lasagne", "spaghetti", "pizza"],
}


def discern_personal_messages(input_msg: str) -> Union[str, List[Any]]:
    """Answer personal question with hard-coded answers to avoid changing personality."""
    personal_pronouns = ["ty", "toba", "tobie", "ci", "ciebie", "cie", "twjj"]
    input_msg = tokenize(input_msg)
    if "co tam" in input_msg or "co slychac" in input_msg:
        return "U mnie spoko, a u Ciebie?"
    if "jak masz na" in input_msg:
        return (
            "Nazywam się "
            + personal_data["nazwisko"]
            + " "
            + personal_data["imię"]
            + ". "
        )
    if "lubisz jesc" in input_msg or "ulubion" in input_msg:
        return "Moim ulubionym daniem jest " + random.choice(
            personal_data["ulubione_potrawy"]
        )
    if "ile masz lat" in input_msg:
        return "Mam " + personal_data["wiek"] + "lata."
    return []


## discord async methods

### on ready async method

In [41]:
@client.event
async def on_ready():
    print(f"{client.user} dołączył do naszego grona na Discordzie!")


### on member join async method

In [42]:
@client.event
async def on_member_join(member):
    await member.create_dm()
    await member.dm_channel.send(f"Siemaneczko {member.name}, witaj na moim terenie!")


### killing session

it is vital since more sessions causes duplications of the same bots and they all answer.


In [43]:
def kill_bot():
    client.loop.run_until_complete(client.logout())
    for task in asyncio.all_tasks(loop=client.loop):
        if task.done():
            task.exception()
            continue
        task.cancel()
        try:
            client.loop.run_until_complete(asyncio.wait_for(task, 5, loop=client.loop))
            task.exception()
        except (
            asyncio.InvalidStateError,
            asyncio.TimeoutError,
            asyncio.CancelledError,
        ):
            pass


### State handling

#### State predicates

In [44]:
def is_culinary_state(msg: str, last_message: str) -> bool:
    """Determine whether to switch state to culinary."""
    meal_synonyms = ["potrawa", "danie", "wyżerka", "jadło", "pożywienie", "prowiant"]
    meals = [
        "kolację",
        "obiad",
        "śniadanie",
        "podwieczorek",
        "lunch",
        "przekąskę",
        "drugie śniadanie",
        "brunch",
    ]
    co_na_phrases = ["co na " + meal for meal in meals]
    culinary_phrases = ["mam w lodówce", "podaj mi przepis na"] + co_na_phrases
    if last_message == "pytanie_o_składniki":
        return True
    if any(culinary_phrase in msg for culinary_phrase in culinary_phrases):
        return True
    if "przepis" in msg:
        return True
    if any(meal_synonym in msg for meal_synonym in meal_synonyms):
        return True
    return False


In [45]:
def is_about_to_die(msg: str) -> bool:
    """Determine whether to switch state to dead."""
    if msg == "nadszedł czas by umierać" or msg == "kill":
        return True
    return False


In [46]:
def is_about_state(msg: str) -> bool:
    """Determine whether to switch state to about."""
    chatbot_terms = [
        "chatbot",
        "czatbot",
        "chat bot",
        "agent konwersacyjny",
        "program komputerowy",
        "chatterbot",
        "program konwersacyjny",
    ]
    questions_about = ["powiedz mi coś o sobie", "kto cię stworzył?", "czym ty jesteś?"]
    return (
        True
        if any(chatbot_term in msg for chatbot_term in chatbot_terms)
        or msg == any(questions_about)
        else False
    )


#### Determining state

In [47]:
def determine_state(msg: str, last_message: str) -> None:
    """Determine the chatbots state basing on users behavior"""
    global state
    if is_culinary_state(msg, last_message):
        state = "culinary"
    elif is_about_to_die(msg):
        state = "dead"
    elif is_about_state(msg):
        state = "about"
    else:
        state = "usual_conversation"


#### Handling state behaviour


In [48]:
def handle_culinary_state(ingredients: List[str], times_of_execution: int) -> str:
    """Handle answering during being in culinary state."""
    encourage_phrases = [
        "Mamma mia!",
        "Va bene!",
        "Już przeszukuję księgi!",
        "Zaraz coś wymyślę...",
        "Zabieram się do pracy!",
        "Muszę zapytać mojego kuchcika, sekunda!",
        "Daj mi chwilkę, a będzie palce lizać!",
        "Noo, ja bym zrobił na przykład: ",
        "Gdybym to ja przygotowywał danie z podanych składników to wybrałbym chyba ten przepis: ",
        "Może to bardzo łatwe, ale na pewno będzie smakować: ",
        "A pomyślałeś może o tym przepisie?",
        "A co powiesz na to: ",
        "Hmmmmm...",
    ]
    ending_phrases = [
        "Mam nadzieję, że będzie smakować!",
        "Smacznego!",
        "Bon Appetit!",
        "Daj mi znać jak poszło.",
        "Buon appetito!",
        "Koniecznie mi powiedz czy wyszło!",
        "Aż samemu chce się gotować!",
        "Chyba sam też spróbuję się za to wziąć.",
        "Wygląda ciekawie, hmmm, może będzie równie apetyczne!",
        "Oby apetyt dopisał!",
        "Życzę dużo przyjemności z gotowania!",
        "Pamiętaj proszę o umyciu rąk zanim zaczniesz gotować!",
        "Nadchodzi pyszność! Czuję to w kościach!",
        "Trzymam kciuki!",
    ]
    if times_of_execution == 0:
        return random.choice(encourage_phrases)
    elif times_of_execution == 1:
        ingredients = (", ").join(ingredients)
        return find_recipes.find_best_recipe(ingredients)
    return random.choice(ending_phrases)


In [49]:
def handle_about_state() -> str:
    """Tell basic information about creation."""
    basic_information = """Jestem botem konwersującym stworzonym na potrzeby
     pracy inżynierskiej Pauliny Landkocz i Zofii Kochutek w roku 2021. 
     Być może nie jestem jeszcze najlepszą możliwą wersją siebie, 
     ale zapewniam, że dziewczyny ciężko pracują, abym się nią stał."""
    return basic_information


### on message async method


In [51]:
last_message = ""
state = "usual_conversation"
usual_conversation_transitions = ["usual_conversation", "culinary", "about", "dead"]
culinary_transitions = ["culinary", "usual_converation", "about", "dead"]
about_transitions = ["usual_conversation", "culinary", "dead"]


@client.event
async def on_message(message):
    global state
    global last_message

    if (
        message.author == client.user
    ):  # checking if the author of the message is the bot
        return
    input_message = message.content.lower()
    subjects_in_input, verbs_in_input, is_sec_sig = find_all_subjects_and_verbs(
        input_message
    )
    important_words = subjects_in_input + verbs_in_input
    determine_state(input_message, last_message)
    if state == "usual_conversation":
        last_message = "usual_conversation"
        if is_greeting(input_message):
            await message.channel.send(
                process_greeting(input_message, is_sec_sig, important_words)
            )
        elif discern_personal_messages(input_message):
            await message.channel.send(discern_personal_messages(input_message))
        elif is_question(input_message):
            if is_sec_sig:
                await message.channel.send(
                    (sentence_similarity.generate_answer(input_message)).capitalize()
                )
            else:
                for i in range(2):
                    await message.channel.send(answer_the_question(input_message, i))
        else:
            await message.channel.send(
                sentence_similarity.similarity_by_embeddings(
                    input_message, important_words
                )
            )

    elif state == "culinary":
        ingredients_msg = (
            "Od teraz rozmawiamy jak kucharz z kucharzem. Proszę podać składniki."
        )
        if last_message == "pytanie_o_składniki":
            last_message = "po_przepisie"
            for i in range(3):
                msg = handle_culinary_state(subjects_in_input, i)
                await message.channel.send(msg)
        else:
            last_message = "pytanie_o_składniki"
            await message.channel.send(ingredients_msg)

    elif state == "dead":
        last_message = ""
        await message.channel.send(
            "Na mnie chyba już pora, jak wiadomo boty idą do nieba."
        )
        print("Terminating")
        raise SystemExit

    elif state == "about":
        last_message = "about"
        await message.channel.send(handle_about_state())


## starting bot

In [52]:
def run_bot():
    try:
        asyncio.get_event_loop().create_task(client.start(TOKEN))
    except SystemExit:
        kill_bot()


run_bot()
